In [1]:
import sys
sys.path.append('..')

In [8]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.llama_cpp import LlamaCPP

import os

model = LlamaCPP(
    model_path=os.path.join("/mnt/d/wsl/llamacpp/models", "neural-chat-7b-v3-3.Q5_K_M.gguf"),
    temperature=0.1,
    max_new_tokens=512,
    # callback_manager=callback_manager,
    # verbose=True, # Verbose is required to pass to the callback manager
    # echo=True,
    context_window=1024*32,
    model_kwargs={"n_gpu_layers": -1}
)


ImportError: cannot import name 'LlamaCPP' from 'llama_index.llms.openai' (/home/digga/micromamba/envs/ll-dm/lib/python3.10/site-packages/llama_index/llms/openai/__init__.py)

In [3]:
import fnmatch

def find_files_with_extension(root_dir, extension):
    
    file_list = []
    for root, _, files in os.walk(root_dir):
        for filename in fnmatch.filter(files, f'*.{extension}'):
            file_list.append(os.path.join(root, filename))
    return file_list

documents = SimpleDirectoryReader(input_files=find_files_with_extension("srd", "json"))

embed_model="local:BAAI/bge-small-en-v1.5"

index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

ImportError: cannot import name 'VectorStoreIndex' from 'llama_index' (unknown location)